In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from graphviz import Digraph
import math

In [2]:
class Value:
    def __init__(self, data, _children = (), _op = '', label = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data = {self.data}, label = {self.label}, grad = {self.grad})\n"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only supporting int/float powers"
        out = Value(self.data**other, (self, ), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other - 1))* out.grad
        
        out._backward = _backward
        return out

    def __radd__(self, other):
        return self + other
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * other ** (-1)
    
    def __neg__(self):
        return self * (-1)
    
    def __sub__(self, other):
        return self + (-other)

    def tanh(self):
        x = self.data
        t = (math.exp(x*2) - 1)/(math.exp(x*2) + 1)
        out = Value(t, (self, ), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ) , 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
        print(topo)

In [3]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format = 'svg', graph_attr = {'rankdir': 'LR'})

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "{%s | data %.4f | grad %.4f}" % (n.label,n.data, n.grad), shape = 'record')
        if n._op:
            dot.node(name = uid + n._op, label = n._op)
            dot.edge(uid + n._op, uid)
        
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2))+ n2._op)
    
    return dot

In [4]:
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')

w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

b = Value(6.881373587019543, label = 'b')

x1w1 = x1*w1
x1w1.label = 'x1w1'
x2w2 = x2*w2
x2w2.label = 'x2w2'

x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1w1x2w2'

n = x1w1x2w2 + b
n.label = 'n'
o = n.tanh()
o.label = 'o'
o.grad = 1.0

In [72]:
o

Value(data = 0.7071067811865476, label = o, grad = 1.0)

In [73]:
o.backward()

[Value(data = 0.0, label = x2, grad = 0.4999999999999999)
, Value(data = 1.0, label = w2, grad = 0.0)
, Value(data = 0.0, label = x2w2, grad = 0.4999999999999999)
, Value(data = 2.0, label = x1, grad = -1.4999999999999996)
, Value(data = -3.0, label = w1, grad = 0.9999999999999998)
, Value(data = -6.0, label = x1w1, grad = 0.4999999999999999)
, Value(data = -6.0, label = x1w1x2w2, grad = 0.4999999999999999)
, Value(data = 6.881373587019543, label = b, grad = 0.4999999999999999)
, Value(data = 0.8813735870195432, label = n, grad = 0.4999999999999999)
, Value(data = 0.7071067811865476, label = o, grad = 1.0)
]


In [74]:
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b; d.label = 'd'
e = a + b; e.label = 'e'
f = d * e; f.label = 'f'

f.backward()

[Value(data = -2.0, label = a, grad = -3.0)
, Value(data = 3.0, label = b, grad = -8.0)
, Value(data = -6.0, label = d, grad = 1.0)
, Value(data = 1.0, label = e, grad = -6.0)
, Value(data = -6.0, label = f, grad = 1.0)
]


In [15]:
from typing import Any


class Neuron:

    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
    
    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:

    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [16]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data = 0.5489130540951652, label = , grad = 0.0)

In [36]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0]
ypred = [n(x) for x in xs]

In [30]:
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data = 2.4332985795822015, label = , grad = 0.0)

In [31]:
loss.backward()

[Value(data = 1.0, label = , grad = 1.8342698268085174)
, Value(data = 0.8181974149128077, label = , grad = 7.101862790523207)
, Value(data = 1.0, label = , grad = -0.6292774365877195)
, Value(data = 0.3801756549685169, label = , grad = -8.200234478992483)
, Value(data = 0.3801756549685169, label = , grad = -1.6552281251145111)
, Value(data = 1.0, label = , grad = 0.526937588786404)
, Value(data = -0.3183474113273357, label = , grad = -0.1621715091367999)
, Value(data = -0.3183474113273357, label = , grad = -1.6552281251145111)
, Value(data = -0.4672960699515954, label = , grad = -7.717111500799394)
, Value(data = 0.5, label = , grad = 0.055884214579247186)
, Value(data = -0.033762243240871126, label = , grad = -11.857118163971414)
, Value(data = -0.016881121620435563, label = , grad = -1.6552281251145111)
, Value(data = -0.484177191572031, label = , grad = -1.6552281251145111)
, Value(data = -0.8025246028993667, label = , grad = -1.6552281251145111)
, Value(data = -0.42234894793084976

In [32]:
for p in n.parameters():
    p.data += -0.01 * p.grad

In [39]:
for k in range(20):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    for p in n.parameters():
        p.grad = 0.0

    loss.backward()
    for p in n.parameters():
        p.data += -.05*p.grad
    
    print(k, loss.data)

[Value(data = 0, label = , grad = 1.0)
, Value(data = 2.1110867728101153, label = , grad = 5.546894241439036e-16)
, Value(data = -0.31993790575921155, label = , grad = -6.833947977521994e-14)
, Value(data = 3.0, label = , grad = 5.0119925333437975e-15)
, Value(data = -0.9598137172776346, label = , grad = -1.5665516474049414e-14)
, Value(data = 0.8847012158665408, label = , grad = -3.7008446577024893e-14)
, Value(data = 0.7144675727919704, label = , grad = -5.267396465875445e-14)
, Value(data = 2.0, label = , grad = -1.1192503531746711e-14)
, Value(data = 1.4289351455839407, label = , grad = -1.5665516474049414e-14)
, Value(data = 2.313636361450482, label = , grad = -1.5665516474049414e-14)
, Value(data = 1.3538226441728471, label = , grad = -1.5665516474049414e-14)
, Value(data = -1.0, label = , grad = -2.1028652196789443e-14)
, Value(data = 1.3423529464620134, label = , grad = 3.7008454195702455e-14)
, Value(data = -1.3423529464620134, label = , grad = -1.5665516474049414e-14)
, Value

In [34]:
ypred

[Value(data = 0.999998077390375, label = , grad = -3.845219249898335e-06),
 Value(data = -0.9999999807414784, label = , grad = 3.8517043110530835e-08),
 Value(data = -0.9999999807407713, label = , grad = 3.85184573126196e-08),
 Value(data = 0.9999980764092439, label = , grad = -3.847181512250941e-06)]